In [2]:
pip install langchain-couchbase

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Load in Couchbase credentials
from dotenv import load_dotenv
import os
load_dotenv()

True

In [7]:
# We create a connection to the Couchbase cluster and then pass cluster object to the Vector store

from datetime import timedelta
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

auth = PasswordAuthenticator(os.getenv("CB_USERNAME"), os.getenv("CB_PASSWORD"))
options = ClusterOptions(auth)
cluster = Cluster(os.getenv("CB_HOSTNAME"), options)

# Wait until cluster is ready to use
cluster.wait_until_ready(timedelta(seconds=5))

In [8]:
# Set Bucket, Scope and Collection for vector search 
BUCKET = "properties"
SCOPE = "2025-listings"
COLLECTION = "united-states"
SEARCH_INDEX = "properties-index"

In [ ]:
# choose AWS for embeddings model
pip install langchain-aws

Note: you may need to restart the kernel to use updated packages.


In [16]:
# create vector store object, with cluster info and search index name

from langchain_aws import BedrockEmbeddings
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")

from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore

vector_store = CouchbaseSearchVectorStore(
    cluster = cluster,
    bucket_name = BUCKET,
    scope_name = SCOPE,
    collection_name = COLLECTION,
    embedding = embeddings,
    index_name = SEARCH_INDEX,
    text_key = "description",
    embedding_key = "embedding",
)

In [17]:
results = vector_store.similarity_search(
    "A beautiful beachside property with a nice backyard and close to good schools",
    k=2,
)
print(results)

[Document(id='56c1bbbc-3fb3-46ae-8c41-c6b4d0ddd7f2', metadata={'HOA_fees': '$0', 'address': '468 2nd St, Encinitas, CA 92024', 'bathrooms': 2, 'bedrooms': 3, 'date_built': 1965, 'geo.lat': 33.037, 'geo.lon': -117.292, 'house_sqft': 1550, 'listed_by': 'Maria Gonzalez DRE #04567890 760-555-0456, Beach Town Properties', 'listing_updated': 'January 11, 2025 at 10:45am', 'lot_sqft': 4800, 'name': 'Encinitas Beach Town Bungalow', 'price': '$945,000', 'url': 'https://www.zillow.com/homedetails/468-2nd-St-Encinitas-CA-92024/234567890_zpid/'}, page_content='Charming 3-bedroom beach bungalow in the heart of Encinitas. Recently updated with modern amenities while preserving original character. Private backyard with fruit trees, walking distance to Moonlight Beach and downtown shops. Perfect for the coastal lifestyle.'), Document(id='0d0f3b4b-01bb-46aa-89d5-7645a1fbe63e', metadata={'HOA_fees': '$0', 'address': '852 Tourmaline St, San Diego, CA 92109', 'bathrooms': 1, 'bedrooms': 2, 'date_built': 1

In [21]:
results = vector_store.similarity_search(
    "A beautiful beachside property with a nice backyard and close to good schools",
    k=2,
)

for i, doc in enumerate(results, 1):
    print(f"Property: {doc.metadata.get('name', 'N/A')}")
    print(f"Price: {doc.metadata.get('price', 'N/A')}")
    print(f"Address: {doc.metadata.get('address', 'N/A')}")
    print(f"Bedrooms: {doc.metadata.get('bedrooms', 'N/A')} | Bathrooms: {doc.metadata.get('bathrooms', 'N/A')}")
    print(f"House Size: {doc.metadata.get('house_sqft', 'N/A')} sqft | Lot Size: {doc.metadata.get('lot_sqft', 'N/A')} sqft")
    print(f"Description: {doc.page_content}")
    print(f"Listed by: {doc.metadata.get('listed_by', 'N/A')}")
    print("-" * 80)

Property: Encinitas Beach Town Bungalow
Price: $945,000
Address: 468 2nd St, Encinitas, CA 92024
Bedrooms: 3 | Bathrooms: 2
House Size: 1550 sqft | Lot Size: 4800 sqft
Description: Charming 3-bedroom beach bungalow in the heart of Encinitas. Recently updated with modern amenities while preserving original character. Private backyard with fruit trees, walking distance to Moonlight Beach and downtown shops. Perfect for the coastal lifestyle.
Listed by: Maria Gonzalez DRE #04567890 760-555-0456, Beach Town Properties
--------------------------------------------------------------------------------
Property: Pacific Beach Cottage
Price: $1,295,000
Address: 852 Tourmaline St, San Diego, CA 92109
Bedrooms: 2 | Bathrooms: 1
House Size: 900 sqft | Lot Size: 5000 sqft
Description: Charming 2-bedroom beach cottage just blocks from the ocean. Original hardwood floors, cozy fireplace, and a large backyard perfect for entertaining. Rare find in this prime location with potential for expansion.
Liste

In [20]:
results = vector_store.similarity_search(
    "A beautiful beachside property with a nice backyard and close to good schools",
    k=2,
)

for i, doc in enumerate(results, 1):
    print(f"\n🏠 {doc.metadata['name']} - {doc.metadata['price']}")
    print(f"📍 {doc.metadata['address']}")
    print(f"🛏️ {doc.metadata['bedrooms']}bd/{doc.metadata['bathrooms']}ba • {doc.metadata['house_sqft']} sqft")
    print(f"📝 {doc.page_content}")
    print("-" * 60)



🏠 Encinitas Beach Town Bungalow - $945,000
📍 468 2nd St, Encinitas, CA 92024
🛏️ 3bd/2ba • 1550 sqft
📝 Charming 3-bedroom beach bungalow in the heart of Encinitas. Recently updated with modern amenities while preserving original character. Private backyard with fruit trees, walking distance to Moonlight Beach and downtown shops. Perfect for the coastal lifestyle.
------------------------------------------------------------

🏠 Pacific Beach Cottage - $1,295,000
📍 852 Tourmaline St, San Diego, CA 92109
🛏️ 2bd/1ba • 900 sqft
📝 Charming 2-bedroom beach cottage just blocks from the ocean. Original hardwood floors, cozy fireplace, and a large backyard perfect for entertaining. Rare find in this prime location with potential for expansion.
------------------------------------------------------------


In [30]:
# With similarity score
results = vector_store.similarity_search_with_score(
    "2 bedroom coastal retreat minutes from La Jolla Cove",
    k=2,
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n🏠 {doc.metadata['name']} - {doc.metadata['price']} (Score: {score:.3f})")
    print(f"📍 {doc.metadata['address']}")
    print(f"🛏️ {doc.metadata['bedrooms']}bd/{doc.metadata['bathrooms']}ba • {doc.metadata['house_sqft']} sqft")
    print(f"📝 {doc.page_content}")
    print("-" * 60)


🏠 La Jolla Coastal Retreat - $875,000 (Score: 0.820)
📍 7654 Prospect St, La Jolla, CA 92037
🛏️ 2bd/2ba • 1350 sqft
📝 Elegant 2-bedroom coastal retreat just minutes from La Jolla Cove. Completely renovated with designer finishes, ocean glimpses from master bedroom, and private courtyard. Perfect for those seeking a luxurious yet low-maintenance lifestyle near world-class beaches and dining.
------------------------------------------------------------

🏠 Ocean View Villa - $4,200,000 (Score: 0.510)
📍 7890 Coast Blvd, La Jolla, CA 92037
🛏️ 5bd/6ba • 4800 sqft
📝 Magnificent 5-bedroom villa with breathtaking ocean views in La Jolla. Private gated entrance, infinity pool, wine cellar, and multiple outdoor entertaining areas. Recently renovated with the finest materials and fixtures.
------------------------------------------------------------


In [28]:
# Load New Properties into Vector Store
from uuid import uuid4
from langchain_core.documents import Document

property_one = Document(
    page_content="Stunning waterfront condo in Mission Bay with panoramic views. Modern 2-bedroom unit with floor-to-ceiling windows, granite countertops, and private balcony. Building amenities include pool, gym, and concierge. Walking distance to beaches and restaurants.",
    metadata={
        "name": "Mission Bay Waterfront Condo",
        "price": "$1,150,000",
        "bedrooms": 2,
        "bathrooms": 2,
        "house_sqft": 1400,
        "lot_sqft": 0,
        "HOA_fees": "$750",
        "date_built": 2021,
        "address": "1234 Bay View Dr, San Diego, CA 92109",
        "listed_by": "John Smith DRE #12345678 858-555-0123, Bay Properties",
        "geo.lat": 32.790,
        "geo.lon": -117.235
    }
)

property_two = Document(
    page_content="Charming Spanish-style home in Kensington with original tile work and arched doorways. 3-bedroom house featuring updated kitchen, hardwood floors, and mature landscaping. Quiet residential street with easy access to downtown.",
    metadata={
        "name": "Kensington Spanish Revival",
        "price": "$875,000",
        "bedrooms": 3,
        "bathrooms": 2,
        "house_sqft": 1650,
        "lot_sqft": 5200,
        "HOA_fees": "$0",
        "date_built": 1935,
        "address": "5678 Adams Ave, San Diego, CA 92116",
        "listed_by": "Maria Lopez DRE #87654321 619-555-0456, Historic Homes Realty",
        "geo.lat": 32.763,
        "geo.lon": -117.115
    }
)

documents = [property_one, property_two]

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)
print(f"Added {len(documents)} properties with auto-generated embeddings")

# delete from vector store
# vector_store.delete(ids=[uuids[-1]])

Added 2 properties with auto-generated embeddings


In [31]:
# Test newly added properties

# With similarity score
results = vector_store.similarity_search_with_score(
    "Spanish-style home in Kensington",
    k=2,
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n🏠 {doc.metadata['name']} - {doc.metadata['price']} (Score: {score:.3f})")
    print(f"📍 {doc.metadata['address']}")
    print(f"🛏️ {doc.metadata['bedrooms']}bd/{doc.metadata['bathrooms']}ba • {doc.metadata['house_sqft']} sqft")
    print(f"📝 {doc.page_content}")
    print("-" * 60)

results = vector_store.similarity_search_with_score(
    "waterfront condo in Mission Bay with panoramic views",
    k=2,
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n🏠 {doc.metadata['name']} - {doc.metadata['price']} (Score: {score:.3f})")
    print(f"📍 {doc.metadata['address']}")
    print(f"🛏️ {doc.metadata['bedrooms']}bd/{doc.metadata['bathrooms']}ba • {doc.metadata['house_sqft']} sqft")
    print(f"📝 {doc.page_content}")
    print("-" * 60)


🏠 Kensington Spanish Revival - $875,000 (Score: 0.763)
📍 5678 Adams Ave, San Diego, CA 92116
🛏️ 3bd/2ba • 1650 sqft
📝 Charming Spanish-style home in Kensington with original tile work and arched doorways. 3-bedroom house featuring updated kitchen, hardwood floors, and mature landscaping. Quiet residential street with easy access to downtown.
------------------------------------------------------------

🏠 Kitsilano Heritage Character Home - $1,295,000 (Score: 0.284)
📍 2234 W 4th Ave, Vancouver, BC V6K 1N6
🛏️ 4bd/3ba • 2450 sqft
📝 Charming 4-bedroom heritage character home in sought-after Kitsilano. Original hardwood floors, leaded glass windows, and period details throughout. Large kitchen with eating area, separate dining room, and beautiful garden. Walking distance to beaches and 4th Avenue shopping.
------------------------------------------------------------

🏠 Mission Bay Waterfront Condo - $1,150,000 (Score: 0.899)
📍 1234 Bay View Dr, San Diego, CA 92109
🛏️ 2bd/2ba • 1400 sqft
📝 